In [ ]:
#default_exp dataloader

# 10-process-data: Importing, cleaning, testing, and saving

**Purpose** The purpose of this notebook is to load, clean, and save the data.  The data should also be tested upon entry and exit to ensure that all of its variables have values that are expected.  You may also form a train and test set here if desired.

**A note on imputation**: If the data requires parameterized imputation and scikit-learn is the expected methodology of interest, evaluate whether performing imputation here or as a pipeline step would be most valuable and helpful.

## YOUR ATTENTION, PLEASE

This notebook is built into a module by nbdev. If you are making changes to it, you should make sure you have nbdev installed (`pip install nbdev`). When your changes are completed, you can then call `nbdev_build_lib` from the root directory of the repository. This will build the notebook into a library, stored in the folder `digital_connoisseur`. 

*If you're using colab, run this cell to ensure you are able to import fastai, then restart the runtime when prompted.*

In [ ]:
#hide
!pip install --upgrade fastai fastcore

#### Helpful packages and preliminaries

In [ ]:
#export
#Import modules for data access and processing
try:
    import pandas as pd
except:
    print("pandas is unavailable")
try:
    import numpy as np
except:
    print("numpy is unavailable")
try:
    from fastai.vision.all import *
except:
    print("fastai.vision.all is unavailable")
try:
    from google.colab import drive
except:
    print("google.colab is unavailable")
try:
  from PIL import Image
except:
  print('PIL is unavailable')
try:
  from tqdm import tqdm
except:
  print('tqdm is unavailable')
from os import system

### Find the data

If you're working on a local machine, you can run the top cell and tell it where your data folder is. If, like me, you are doing this from colab, you'll need to upload the data to google drive. Compress the data into a zip file, title it `dc-data.zip`, upload it to drive (don't put it in any folders), then run the cell; it will ask you where the zip file is, copy the zip file into the current working directory, and then unzip the folder.  Also, in order to be able to access other files in the repository, you can upload the repository to google drive, keep the drive mounted, and use that. You will need to `cd` to the directory where the repository is on Google Drive.

In [ ]:
#export
### IF ON LOCAL MACHINE ###

def get_path_local():
    return input('Enter path to data\n> ')

In [ ]:
#export
### IF USING GOOGLE DRIVE ###

def get_path_zip():
    path_to_zip = input('Enter path to zip file\n>')
    path_to_folder = input('Where do you want it stored?\n>')

    print(f'Moving zip file from {path_to_zip} to {path_to_folder}/dc-data.zip...')
    system(f'cp "{path_to_zip}" "{path_to_folder}/dc-data.zip"')
    
    copied_zip = f'{path_to_folder}/dc-data.zip'

    print(f'Unzipping zip file at {copied_zip}...')
    system(f'unzip {copied_zip} -d {path_to_folder}')

    print(f'Done. Data should be at {path_to_folder}/dc-data.')

    return f'{path_to_folder}/dc-data'

### Get data into `DataLoader`

The purpose of the above scripts was to locate where the data is stored. Now, we need to actually load it, using `DataLoader`. This part of the pipeline is subject to change, as we revise and improve our data processing.

In [ ]:
#export
# define the functions we need
# make the segment:
def make_segment(left, up, std_length, std_width, img):
    width, height = img.size
    right = left + std_width
    below = up + std_length

    right_flag = False
    below_flag = False

    if right > width:
        right = width
        left = width - std_width
        right_flag = True

    if below > height:
        below = height
        up = below - std_length
        below_flag = True

    box=(left, up, right, below)
    sample_img = img.crop(box)
    return sample_img, right_flag, below_flag


# get the left and up
def get_left_up(right_flag,below_flag,pre_left,pre_up,overlap):
    if right_flag==True and below_flag==False :
        # reach the right edge
        next_left=0
        nex_up=pre_up+overlap

    if right_flag==False and below_flag==True :
        # reach the below edge
        next_left=pre_left+overlap
        nex_up=pre_up

    if right_flag==True and below_flag==True:
        #reach both side
        next_left = -100
        nex_up = -100

    if right_flag==False and below_flag==False:
        next_left = pre_left + overlap
        nex_up = pre_up

    return next_left,nex_up

    #get precent of white space of sample image:
def get_white_precent(sample_img,std_width,std_height):
    sample_img=list(sample_img.getdata())
    count=0

    for x in sample_img:
        if x[0]==255 and x[1]==255 and x[2]==255:
            count=count+1

    result=int((count/(std_width*std_height))*100)

    return result

In [ ]:
#export
imagesAndLabels = None
imagesAndLabelsDict = None

def set_labels_dict(path_to_data):
    global imagesAndLabels
    global imagesAndLabelsDict
    
    imagesAndLabels = pd.read_csv(path_to_data+'/MocheThemeRollouts.csv')
    imagesAndLabelsDict = {}

    # Read the images and labels into the dictionary
    for rownum in range(len(imagesAndLabels)):
      image = imagesAndLabels.iloc[rownum,0]
      label = imagesAndLabels.iloc[rownum,1].strip()
      imagesAndLabelsDict[image] = label

# Define a function that looks up the image name (minus the ".jpg" suffix) in the dictionary
def label_func(filename):
  strf = str(filename)
  simplefilename = strf[strf.rindex('/')+1:-4]
  img_id = int(simplefilename)
  return imagesAndLabelsDict[img_id]

In [ ]:
#export

# define attribute
std_length = 178  # size we want
std_width = 178
overlap = int(178 / 2)

def slice_image(img, path_to_data): #This function cuts up each image into overlapping squares
  strf = str(img)
  img_id = strf[strf.rindex('/')+1:-4]
  image = Image.open(img)
  # cut the image
  left = 0
  up = 0
  while left != -100:
    sample_img, right_flag, below_flag = make_segment(left, up, std_length, std_width, image)
    # action to sample img:
    wh_precent = get_white_precent(sample_img, std_width, std_length)
        
    theme = label_func(img)
    folder = theme.replace(' ', '')
        
    if wh_precent < 70:
      filepath = path_to_data + '/' + folder + "/" + 'xxseg-' + str(img_id) + "-" +str(int(left)) + "-" + str(int(up)) + "-" + str(wh_precent) + ".jpg"
      try:
        sample_img.save(filepath)
      except:
        system(f'mkdir {path_to_data}/{folder}')
        sample_img.save(filepath)
        
    # get next left and up:
    pre_left = left
    pre_up = up
    left, up = get_left_up(right_flag, below_flag, left, up, overlap)

def segment_images(path_to_data): #This segments each image and saves the segments to disk
  print('Removing old subdirectories...')
  system(f'rm -R -- {path_to_data}/*/')
  print('Performing segmentation and saving to new subdirectories...')
  try:
    for img in tqdm(get_image_files(path_to_data,recurse=False),desc="Segmenting images"):
      slice_image(img,path_to_data)
  except:
    for img in get_image_files(path_to_data,recurse=False):
      slice_image(img,path_to_data)
  print('Done.')

In [ ]:
#export
from os import sep
from random import shuffle

def get_overlap(img_1, img_2):
  """Get whether img_1 and img_2 overlap with each other.
  The arguments should be the filenames, not the actual images."""
  img_1_split = img_1.split('-')
  img_2_split = img_2.split('-')
  id1 = img_1_split[1]
  id2 = img_2_split[1]
  if id1 != id2:
    return False
  x1 = int(img_1_split[2])
  x2 = int(img_2_split[2])
  y1 = int(img_1_split[3])
  y2 = int(img_2_split[3])
  if (abs(x1-x2) < std_width) and (abs(y1-y2) < std_length):
    return True
  return False

training_set = []
holdout_set = []

def get_segmented_image_files(path_to_data):
  files = get_image_files(path_to_data)
  segmented_files = []
  for image in files:
    if 'xxseg' in str(image): #segmented images all contain the string "xxseg" in the title, to identify them
      segmented_files.append(image)
  return segmented_files

def sort_segmented_image_files(path_to_data):
  """Sort segmented images into training_set and holdout_set"""
  global training_set, holdout_set
  files = get_image_files(path_to_data)
  segmented_files = get_segmented_image_files(path_to_data)
  shuffle(segmented_files)

  holdout_ids = {} # IDs of pictures we are holding out, sorted by label
  holdout_segments = {} # Names of segments we are holding out, sorted by label

  for img in segmented_files:
    full_path = str(img)
    path_split = full_path.split(os.sep)
    name = path_split[-1]
    label = path_split[-2]

    if label not in holdout_ids.keys():
      holdout_ids[label] = None

    if label not in holdout_segments.keys():
      holdout_segments[label] = []

    if label == 'BadmintonTheme' or label == 'RevoltoftheObjects':
      if len(holdout_segments[label]) < 75:
        holdout_segments[label].append(img)
      else:
        overlap = False
        for holdout_img in holdout_segments[label]:
          img_name = str(holdout_img).split(os.sep)[-1]
          if get_overlap(name, img_name):
            overlap = True
        if not overlap:
          training_set.append(img)
    
    else:
      id = name.split('-')[1]
      if holdout_ids[label] == None:
        holdout_ids[label] = id
      
      if holdout_ids[label] == id:
        holdout_segments[label].append(img)
      else:
        training_set.append(img)

  for label in holdout_segments.keys():
    holdout_set.extend(holdout_segments[label])

def get_training_files(path_to_data):
  return training_set

def get_holdout_files(path_to_data):
  return holdout_set

In [ ]:
#hide
#Use this debugging code to check for overlaps between training and holdout sets
for img in tqdm(training_set, desc="Checking for overlaps"):
  for holdout_img in holdout_set:
    img_name = str(img).split(os.sep)[-1]
    holdout_name = str(holdout_img).split(os.sep)[-1]
    if get_overlap(img_name, holdout_name):
      print(str(img), str(holdout_img))
    assert not get_overlap(img_name, holdout_name)
print('\nNo overlaps detected')

In [ ]:
#hide
# Use this debugging code to check how many of each label are in the training and holdout sets
label_counts = {}
for img in holdout_set:
  label = str(img).split(os.sep)[-2]
  try:
    label_counts[label] = label_counts[label] + 1
  except KeyError:
    label_counts[label] = 1

print(label_counts)

label_counts = {}
for img in training_set:
  label = str(img).split(os.sep)[-2]
  try:
    label_counts[label] = label_counts[label] + 1
  except KeyError:
    label_counts[label] = 1

print(label_counts)

In [ ]:
#export
#these are global that help get_dls know what work it's already done.
global_path_to_data = None
already_set_labels = False
already_segmented = False
already_sorted = False

In [ ]:
#export

def get_dls(colab=False, force_reset=False, path_to_data=None, skip_segmentation=False):
  global global_path_to_data
  global already_set_labels
  global already_segmented
  global already_sorted

  if force_reset:
    global_path_to_data = None
    already_set_labels = False
    already_segmented = False
    already_sorted = False

  if skip_segmentation:
    print('Skipping segmentation.')
    print("Errors will be thrown if segmentation hasn't already been performed.")
    already_segmented = True

  if path_to_data:
    global_path_to_data = path_to_data

  if global_path_to_data == None:
    if colab:
        path_to_data = get_path_zip()
        global_path_to_data = path_to_data
    else:
        path_to_data = get_path_local()
        global_path_to_data = path_to_data
  else:
    path_to_data = global_path_to_data
      
  path = Path(path_to_data)

  if not already_set_labels: 
    set_labels_dict(path_to_data)
    
  if not already_segmented:
    segmented_images = get_segmented_image_files(path_to_data)

    if len(segmented_images) != 0:
      print('Segmentation seems to have already been performed on this data.')
      repeat_segmentation = input('Do you want to do it again? y/[n]\n> ')
      if repeat_segmentation != 'y' and repeat_segmentation != 'Y':
        already_segmented = True
    else:
      already_segmented = False

  if not already_segmented:
    segment_images(path_to_data)
    already_segmented = True

  if not already_sorted:
    sort_segmented_image_files(path_to_data)

  print('Constructing DataBlock...')
  drawings = DataBlock(
      blocks=(ImageBlock, CategoryBlock), 
      get_items=get_training_files, 
      splitter=RandomSplitter(valid_pct=0.2, seed=42),
      get_y=parent_label,
      item_tfms=Resize(224),
      batch_tfms=Rotate(draw=lambda x: x.new_empty(x.size(0)).uniform_(-180, 180))
  )

  print('Done.')
  return drawings.dataloaders(path)